# UFO Question 

Our data science team has predicted that the Earth is going to be invaded by an alien force in the
next years. Our only hope is to replicate a device that can block all alien technology in a radius of
~300km. Sadly, the device was sold in 2004 to an anonymous buyer to protect her hometown and
we don't know how contact her again. We know that the device has been active since 2004 in one
city in the USA, and we want to know where to start our search.
We've included a dataset called ​ufo.csv​. This dataset contains over 80,000 reports of UFO sightings
over the last century (all of them verified by the ESA). Using this dataset, try to guess the city in
which the device has been hidden.


In [1]:
%pip install folium

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pymongo import MongoClient

import pandas as pd

import re
import folium

In [3]:
cliente=MongoClient()

db=cliente.UFO
colect = db.UFO

In [4]:
df = pd.DataFrame(list(colect.find({}
                                  
                                  )))

df.head()

,_id,,datetime,city,state,country,shape,duration,total_time,comments,date_posted,latitude,longitude,year,distance
0,624f10f03482f6569b33dc32,0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.9411111,2004,1242.66777234
1,624f10f03482f6569b33dc33,1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082,2005,1325.48631876
2,624f10f03482f6569b33dc34,2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667,2008,6515.41657656
3,624f10f03482f6569b33dc35,3,10/10/1956 21:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.6458333,2004,1211.9713516
4,624f10f03482f6569b33dc36,4,10/10/1960 20:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.8036111,2004,6960.92339627


In [5]:
df.datetime=df.datetime.str.extract(r'(\d{4})')

In [10]:
df.information()

AttributeError: 'DataFrame' object has no attribute 'information'

In [8]:
df[(df.country == 'us') & (df.datetime > 2004) & (df.distance < 305) ]

TypeError: '>' not supported between instances of 'str' and 'int'